In [1]:
import scanpy as sc
import numpy as np
import os
import anndata2ri
import pathlib
import scvelo as scv
from scipy import io
import anndata#
import pandas as pd
from tqdm import tqdm
import argparse
import sys
import re
# Activate the anndata2ri conversion between SingleCellExperiment and AnnData
anndata2ri.activate()

#Loading the rpy2 extension enables cell magic to be used
#This runs R code in jupyter notebook cells
%load_ext rpy2.ipython

sc.settings.verbosity = 3
# sc.logging.print_versions()

import warnings
warnings.filterwarnings("ignore")
PROJECT = "SBharadwaj_20240318"
outdir = os.path.join("/media/hieunguyen/HD01/outdir/CRC1382", PROJECT)

path_to_main_src = "/home/hieunguyen/CRC1382/src_2023/SBharadwaj/deep_seq_batch"
samplesheet = pd.read_csv(os.path.join(path_to_main_src, "SampleSheet_for_DGE_and_CellChat.csv"))
samplesheet["full_dataset_name"] = samplesheet[["PROJECT", "dataset_name"]].apply(
    lambda x: f"{x[0]}_{x[1]}", axis = 1
)
samplesheet = samplesheet[~samplesheet["full_dataset_name"].duplicated()]


In [8]:
#####------------------------------------------------------------#####
##### Read anndata
#####------------------------------------------------------------#####
full_name = "SBharadwaj_20240318_Sample_1_4_7_8_2_5_full"

project = samplesheet[samplesheet["full_dataset_name"] == full_name]["PROJECT"].values[0]
dataset_name = samplesheet[samplesheet["full_dataset_name"] == full_name]["dataset_name"].values[0]
path_to_input_s_obj = samplesheet[samplesheet["full_dataset_name"] == full_name]["path"].values[0]
path_to_main_output = os.path.join(outdir, project, "data_analysis")

path_to_seurat2anndata = os.path.join(path_to_main_output, "09_output", "seurat2anndata")

adata = sc.read_h5ad(os.path.join(path_to_seurat2anndata, '{}.h5ad'.format(full_name)))
obsdf = adata.obs.copy()
obsdf["barcode"] = obsdf[["barcode", "name"]].apply(lambda x: x[0].replace(f"{x[1]}_{x[1]}", f"{x[1]}"), axis = 1) 

adata.obs.index = obsdf.barcode.values
adata.var_names_make_unique()

colordf = pd.read_csv(os.path.join(path_to_seurat2anndata, 'colordf_{}.csv'.format(full_name)))
colors = colordf["color"].values

#####------------------------------------------------------------#####
##### READ LOOM DATA
#####------------------------------------------------------------#####
path_to_loom_data = "/media/hieunguyen/HD01/outdir/CRC1382/SBharadwaj_20240318/loom"
all_looms = [item for item in pathlib.Path(path_to_loom_data).glob("*.loom")]

velodata_dict = dict()
for input_loom in tqdm(all_looms):
    velodata = scv.read_loom(input_loom)
    samplename = input_loom.name.replace(".loom", "")

    new_obs = ["{}_{}_{}-1".format(samplename, samplename, item.split(":")[1].replace("x", "")) for item in velodata.obs.index]
    velodata.obs.index = new_obs
    velodata.obs["barcode"] = new_obs
    velodata.var_names_make_unique()
    pattern = re.compile('_[A|T|G|C]*-')
    new_obs = [samplename + pattern.search(string = item).group() + item.split("-")[1]
            for item in velodata.obs.index]
    velodata.obs.index = new_obs
    velodata_dict[samplename] = velodata
    velodata.var_names_make_unique()

all_velodata = velodata_dict[list(velodata_dict.keys())[0]]
for data in list(velodata_dict.keys())[1:]:
    all_velodata = all_velodata.concatenate(velodata_dict[data])
    
new_obs = [item.split("-")[0] + "-1" for item in all_velodata.obs.index]
all_velodata.obs.index = new_obs
all_velodata.var_names_make_unique()



100%|██████████| 8/8 [00:25<00:00,  3.17s/it]


UnboundLocalError: local variable 'id_length' referenced before assignment

In [9]:
merge_data = scv.utils.merge(adata, all_velodata)

UnboundLocalError: local variable 'id_length' referenced before assignment